# **PROJET 6** : Analysez les ventes d'une librairie avec Python
---

**Mise en situation** 

L’entreprise Lapage était originellement une librairie physique avec plusieurs points de vente. Mais devant le succès de certains de ses produits et l’engouement de ses clients, elle a décidé depuis 2 ans d’ouvrir un site de vente en ligne. La structure a besoin d’aide pour mieux comprendre ses données, c’est pourquoi elle vous a recruté. Vous intervenez en tant que Data Analyst afin de faire le point sur l’activité.

**Consignes**
- Analyses des indicateurs de ventes : les chiffres clés, KPI, graphiques générés, etc. 
- Analyse des corrélations

**Compétences évaluées**
- Réaliser un test statistique
- Réaliser une analyse bivariée pour interpréter des données
- Analyser des séries temporelles

## Sommaire
---

- [**Section 1: Import**](#section1)
- [**Section 2: Jointures**](#section2)
- [**Section 3: Manipulation DFT**](#section3)
- [**Section 4: Etude du CA**](#section4)
- [**Section 5: Correlations**](#section5)
- [**Section 6: Conclusion**](#section6)
---


<a id="section1"></a>
# **IMPORT**
---

In [ ]:
# import des modules et des packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import statsmodels.api as sm
from scipy.stats import pearsonr
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
from scipy.stats import linregress
#import fonctions as fc

In [ ]:
#Import fichiers csv
customers = pd.read_csv('customersv2.csv', delimiter=';')
transactions = pd.read_csv('Transactionsv2.csv',delimiter=';')
products = pd.read_csv('productsv2.csv',delimiter=';')

## **Fonctions**

In [ ]:
def analyse_statistique(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    stats = df.describe(include='all')
    
    # Ajouter la skewness et la kurtosis
    stats.loc['skewness'] = df[numeric_cols].skew()
    stats.loc['kurtosis'] = df[numeric_cols].kurtosis()

    return stats

In [ ]:
def etude_fichier(df):
    '''Cette fonction étudie un dataframe en regardant sa forme, les types, le nombre de valeurs uniques et manquantes'''
    print("Nombre de colonnes :", df.shape[1])
    print()
    print("Le type est : \n", df.dtypes)
    print()
    print('Nombre de valeurs uniques :')
    print(df.nunique())
    print()
    print('Le nombre de valeurs manquantes :\n', df.isnull().sum())

In [ ]:
def identifier_differences_entre_cles(df1, df2, keys_df1=None, keys_df2=None):
    """
    Identifie les différences entre les clés de deux DataFrames en utilisant les colonnes spécifiées pour chaque DataFrame.

    Args:
        df1 (pandas.DataFrame): Le premier DataFrame.
        df2 (pandas.DataFrame): Le deuxième DataFrame.
        keys_df1 (list or str, optional): Les colonnes à utiliser comme clés pour le premier DataFrame. 
                                          Si None, utilise la première colonne.
                                          Par défaut, None.
        keys_df2 (list or str, optional): Les colonnes à utiliser comme clés pour le deuxième DataFrame. 
                                          Si None, utilise la première colonne.
                                          Par défaut, None.
    """
    if keys_df1 is None:
        keys_df1 = [df1.columns[0]]
    if keys_df2 is None:
        keys_df2 = [df2.columns[0]]

    unique_keys_df1 = set(df1[keys_df1].drop_duplicates().apply(tuple, axis=1))
    unique_keys_df2 = set(df2[keys_df2].drop_duplicates().apply(tuple, axis=1))

    keys_only_in_df1 = unique_keys_df1 - unique_keys_df2
    keys_only_in_df2 = unique_keys_df2 - unique_keys_df1

    print("Nombre de clés absentes dans DF1:", len(keys_only_in_df1))
    print("Nombre de clés absentes dans DF2:", len(keys_only_in_df2))

In [ ]:
def etudier_jointures(df1, df2):
    """ Effectue une étude des jointures possibles entre deux DataFrames en utilisant les colonnes des ID comme clés de jointure.

    Args:
        df1 (pandas.DataFrame): Le premier DataFrame.
        df2 (pandas.DataFrame): Le deuxième DataFrame.
    """
    id_columns_df1 = [col for col in df1.columns if 'id' in col.lower()]
    id_columns_df2 = [col for col in df2.columns if 'id' in col.lower()]

    results = {}

    for on_column_df1 in id_columns_df1:
        for on_column_df2 in id_columns_df2:
            # Jointure interne (inner join)
            inner_unique_keys = len(pd.merge(df1, df2, how='inner', left_on=on_column_df1, right_on=on_column_df2).drop_duplicates(subset=on_column_df1))

            # Jointure à gauche (left join)
            left_unique_keys = len(pd.merge(df1, df2, how='left', left_on=on_column_df1, right_on=on_column_df2).drop_duplicates(subset=on_column_df1))

            # Jointure à droite (right join)
            right_unique_keys = len(pd.merge(df1, df2, how='right', left_on=on_column_df1, right_on=on_column_df2).drop_duplicates(subset=on_column_df1))

            # Jointure externe complète (full outer join)
            outer_unique_keys = len(pd.merge(df1, df2, how='outer', left_on=on_column_df1, right_on=on_column_df2).drop_duplicates(subset=on_column_df1))

            results[f"{on_column_df1} - {on_column_df2}"] = {
                'inner': inner_unique_keys,
                'left': left_unique_keys,
                'right': right_unique_keys,
                'outer': outer_unique_keys
            }

    for key, values in results.items():
        print(f"Colonnes de jointure : {key}")
        print("Nombre de clés uniques après jointure interne:", values['inner'])
        print("Nombre de clés uniques après jointure à gauche:", values['left'])
        print("Nombre de clés uniques après jointure à droite:", values['right'])
        print("Nombre de clés uniques après jointure externe:", values['outer'])
        print()

In [ ]:
def detecter_outliers(df, column, seuil=2.0):
    """
    Détecte les outliers dans une colonne d'un DataFrame en utilisant le Z-score.

    Args:
        df (pandas.DataFrame): Le DataFrame contenant les données numériques.
        column (str): Le nom de la colonne à étudier.
        seuil (float): Le seuil pour définir les outliers. Par défaut, 2.0.
    """
    if column not in df.columns:
        raise ValueError(f"La colonne '{column}' n'existe pas dans le DataFrame.")
    
    series = df[column]
    z_scores = np.abs((series - series.mean()) / series.std())
    outliers_mask = z_scores > seuil
    outliers_percentage = (outliers_mask.mean() * 100).round(2)
    
    print(f"Le pourcentage de valeurs considérées comme des outliers en utilisant le Z-score au seuil 2 dans la colonne '{column}' est {outliers_percentage}%")

In [ ]:
def plot_skewness_kurtosis(data, column):
    """
    Crée des graphiques pour visualiser la skewness et la kurtosis d'une distribution.

    Args:
        data (pandas.DataFrame): Le DataFrame contenant les données à analyser.
        column (str): Le nom de la colonne à analyser.

    Returns:
        None
    """
    skewness = data[column].skew()
    kurtosis = data[column].kurtosis()

    plt.figure(figsize=(12, 6))

    # Histogramme de skewness
    plt.subplot(1, 2, 1)
    sns.histplot(data[column], kde=True)
    plt.axvline(x=data[column].mean(), color='r', linestyle='dashed', linewidth=2)
    plt.title(f'Skewness: {skewness:.2f}')

    # Histogramme de kurtosis
    plt.subplot(1, 2, 2)
    sns.histplot(data[column], kde=True)
    plt.axvline(x=data[column].mean(), color='r', linestyle='dashed', linewidth=2)
    plt.axvline(x=data[column].mean() + data[column].std(), color='g', linestyle='dashed', linewidth=2)
    plt.axvline(x=data[column].mean() - data[column].std(), color='g', linestyle='dashed', linewidth=2)
    plt.axvline(x=data[column].mean() + 2 * data[column].std(), color='b', linestyle='dashed', linewidth=2)
    plt.axvline(x=data[column].mean() - 2 * data[column].std(), color='b', linestyle='dashed', linewidth=2)
    plt.title(f'Kurtosis: {kurtosis:.2f}')

    plt.tight_layout()

    # Analyse de la répartition (skewness)
    if skewness > 0:
        skewness_analysis = "La distribution est inclinée positivement vers la droite (queue à droite),\n ce qui signifie qu'elle a une longue queue à droite et que \n les valeurs sont plus dispersées à droite de la moyenne."
    elif skewness < 0:
        skewness_analysis = "La distribution est inclinée positivement vers la gauche (queue à gauche),\n ce qui signifie qu'elle a une longue queue à gauche et \n que les valeurs sont plus dispersées à gauche de la moyenne."
    else:
        skewness_analysis = "La distribution est parfaitement symétrique, \n ce qui signifie que les valeurs sont également réparties des deux côtés de la moyenne."

    # Analyse de l'aplatissement (kurtosis)
    if kurtosis > 0:
        kurtosis_analysis = "La distribution est leptokurtique, \n ce qui signifie qu'elle a des pics plus fins et des queues plus épaisses que la distribution normale.\n Les valeurs sont plus concentrées autour de la moyenne, avec des queues plus épaisses indiquant des valeurs extrêmes."
    elif kurtosis < 0:
        kurtosis_analysis = "La distribution est platykurtique, \n ce qui signifie qu'elle a des pics plus larges et des queues plus minces que la distribution normale.\n Les valeurs sont moins concentrées autour de la moyenne, avec moins de valeurs extrêmes."
    else:
     kurtosis_analysis = "La distribution est mésokurtique, \n ce qui signifie qu'elle a un aplatissement similaire à une distribution normale. \nLes pics et les queues sont similaires à une distribution normale."


    print("Analyse de la répartition (skewness):", skewness_analysis)
    print("Analyse de l'aplatissement (kurtosis):", kurtosis_analysis)

    plt.show()

## **Etude fichier csv**

## customers

In [ ]:
customers.head()

In [ ]:
etude_fichier(customers)

In [ ]:
analyse_statistique(customers)

In [ ]:
# ajout d'un entête pour la colonne id
customers.rename(columns={'52' : 'client_id'}, inplace=True)

## transactions

In [ ]:
transactions.head()

In [ ]:
etude_fichier(transactions)

In [ ]:
analyse_statistique(transactions)

## products

In [ ]:
products.head()

In [ ]:
etude_fichier(products)

In [ ]:
products['categ'] = products['categ'].astype(str)

In [ ]:
analyse_statistique(products)

In [ ]:
plot_skewness_kurtosis(products, column='price')

**Skewness (Asymétrie) :**
La skewness est une mesure de l'asymétrie de la distribution des données par rapport à la moyenne (barre rouge). Elle indique la tendance d'une distribution à s'étendre plus d'un côté que de l'autre. Plus précisément, la skewness mesure la proportion de données étalées vers la droite ou la gauche de la moyenne.
- Une skewness positive indique que la distribution a une queue plus longue du côté droit de la moyenne, ce qui signifie qu'il y a des valeurs extrêmement élevées.
- Une skewness négative indique que la distribution a une queue plus longue du côté gauche de la moyenne, ce qui signifie qu'il y a des valeurs extrêmement faibles.

- Une skewness de zéro indique que la distribution est relativement symétrique autour de la moyenne.

**Kurtosis (Aplatissement) :**
La kurtosis mesure la "pointe" ou l'"aplatissement" d'une distribution par rapport à la distribution normale (gaussienne). Une distribution normale a une kurtosis de 3. Si la kurtosis est supérieure à 3, la distribution est plus pointue et étroite que la normale (distribution leptokurtique). Si la kurtosis est inférieure à 3, la distribution est plus plate et large que la normale (distribution platykurtique).

En résumé, la skewness et la kurtosis sont des mesures statistiques qui fournissent des informations sur la forme, l'asymétrie et la concentration des données dans une distribution. Elles sont utiles pour comprendre les caractéristiques de la distribution des données et pour détecter des anomalies dans les données.

In [ ]:
detecter_outliers(products,'price', seuil=2.0)

<a id="section2"></a>
# **JOINTURES**
---

## Tests et études de jointures entre transactions et products

In [ ]:
# Utilisation de la fonction pour étudier les jointures
etudier_jointures(transactions[['id_prod']], products[['id_prod']])

In [ ]:
# Etude du nombre de clés manquantes entre la table transactions et  products
identifier_differences_entre_cles(transactions, products, keys_df1=['id_prod'], keys_df2=['id_prod'])

On garde la jointure left car dans la jointure right on ne garde pas les produits qui n'ont pas eu de transactions.

In [ ]:
# On va créer 2 df, un avec nos produits sans vente et l'autre avec seulement les produits avec des ventes pour regarder le CA.
df_avec_produits_sans_vente = pd.merge(transactions,products, how='right', on='id_prod')
df = pd.merge(transactions,products, how='left', on='id_prod')

In [ ]:
etude_fichier(df)

## Tests et étude de jointure entre df et customers

In [ ]:
# Utilisation de la fonction pour étudier les jointures
etudier_jointures(df[['client_id']], customers[['client_id']])

In [ ]:
# Etude du nombre de clés manquantes entre la table transactions et customers
identifier_differences_entre_cles(df, customers, keys_df1=['client_id'], keys_df2=['client_id'])

In [ ]:
# On va créer 2 df, un avec nos produits sans vente et l'autre avec seulement les produits avec des ventes pour regarder le CA.
dft_avec_produits_sans_vente = pd.merge(df,customers, how='right', on='client_id')
dft = pd.merge(df,customers, how='left', on='client_id')

In [ ]:
etude_fichier(dft)

<a id="section3"></a>
# **MANIPULATION DFT**
---

## Etude DFT

In [ ]:
dft.head()

In [ ]:
etude_fichier(dft)

In [ ]:
analyse_statistique(dft)

## Modifications DFT

In [ ]:
# Conversion de la colonne "birth" en format année
dft['birth'] = pd.to_numeric(dft['birth'], errors='coerce').astype(pd.Int64Dtype()) # Convertir en type entier

# Convertir la colonne "date" en format de date
dft['date'] = pd.to_datetime(dft['date'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

# Calcul de l'âge
current_year = datetime.now().year
dft['age'] = current_year - dft['birth']

In [ ]:
# Définition des intervalles d'âge
intervalles = [0, 18, 30, 40, 50, 60, 70, float('inf')]
tranche = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '70+']

# Création de la colonne "age_group" avec les tranches d'âge
dft['tranche'] = pd.cut(dft['age'], bins=intervalles, labels=tranche, right=False)

In [ ]:
# Conversion de l'age en int
dft['age'] = pd.to_numeric(dft['age'], errors='coerce')

# Conversion de categ en string
dft['categ'] = dft['categ'].astype(str)

In [ ]:
# pour l'attribution des profils je dois me débarsser des nuls qui crée des erreur dans la fonction assign profile
dft.dropna(subset=['age', 'sex'], inplace=True)

In [ ]:
def assign_profile(row):
    age = row['age']
    sex = row['sex']

    # Définir la tranche d'âge
    if age < 18:
        tranche = '0-18'
    elif age < 30:
        tranche = '18-30'
    elif age < 40:
        tranche = '30-40'
    elif age < 50:
        tranche = '40-50'
    elif age < 60:
        tranche = '50-60'
    elif age < 70:
        tranche = '60-70'
    else:
        tranche = '70+'

    # Définir le profil en combinant le sexe et la tranche d'âge
    if sex == 'm':
        profil = 'Homme'
    else:
        profil = 'Femme'

    return profil + ' entre ' + tranche + ' ans'

# Appliquer la fonction d'assignation de profil à chaque ligne du DataFrame
dft['profil'] = dft.apply(assign_profile, axis=1)

In [ ]:
# Calculer le panier moyen par âge
panier_moyen_par_age = round(dft.groupby('age')['price'].mean().reset_index(name='panier_moyen_par_age'))

# Joindre cette information à votre dataframe original
dft = dft.merge(panier_moyen_par_age, on='age', how='left')

In [ ]:
dft.head()

In [ ]:
# Calculer le panier moyen par tranche d'age
panier_moyen_par_tranche_age = round(dft.groupby('tranche')['price'].mean().reset_index(name='panier_moyen_par_tranche_age'))

# Joindre cette information à votre dataframe original
dft = dft.merge(panier_moyen_par_tranche_age, on='tranche', how='left')

In [ ]:
# Calculer la fréquence d'achat pour chaque client
frequence_achat = dft.groupby('client_id').size().reset_index(name='frequence_achat')

# Joindre cette information à votre dataframe original
dft = dft.merge(frequence_achat, on='client_id', how='left')

In [ ]:
dft.head()

In [ ]:
dft.info()

Nous avosn toutes les colonnes qui nous eeront nécessaires pour faire les analyses demandées. L'age des clients, leurs profils ainsi qu'une tranche d'age.

<a id="section4"></a>
# **ETUDE DU CA**
---

##  Evolution dans le temps et  décomposition en moyenne mobile pour évaluer la tendance golbale

In [ ]:
# Regrouper les données par mois et calculer le chiffre d'affaires total par mois
vente_mois = dft.groupby(pd.Grouper(key='date', freq='M')).sum()['price']
vente_mois.head()

In [ ]:
# Calcul de la moyenne mobile avec une fenêtre de 3

'''La moyenne mobile est une méthode couramment utilisée pour lisser les fluctuations temporaires dans les séries de données
En d'autres termes, elle aide à éliminer le "bruit" et à mettre en évidence la tendance sous-jacente.

La "fenêtre" est le nombre de périodes de temps sur lesquelles la moyenne est calculée.
Ici, nous utilisons une fenêtre de 3, ce qui signifie que nous calculons la moyenne des ventes pour chaque groupe de 3 mois consécutifs.

vente_mois.rolling(window=3).mean() crée une nouvelle série de données où chaque point est la moyenne des ventes des 3 mois précédents.
#Note : Les deux premiers points de la série de la moyenne mobile seront NaN parce qu'ils n'ont pas deux mois précédents sur lesquels calculer une moyenne.'''
rolling_mean = vente_mois.rolling(window=3).mean()


In [ ]:
# Calcul de la régression linéaire pour la tendance

'''Utiliser la régression linéaire pour trouver la meilleure ligne droite qui s'adapte aux données de vente mensuelles
Les valeurs x sont les indices des mois (0, 1, 2, ...) et les valeurs y sont les ventes mensuelles'''
slope, intercept, r_value, p_value, std_err = linregress(range(len(vente_mois)), vente_mois.values)

'''Calcul de la ligne de tendance (y = mx + b) en utilisant la pente (m) et l'ordonnée à l'origine (b) calculées précédemment
La liste "trendline" contient les valeurs y de la ligne de tendance pour chaque mois'''
trendline = [slope * x + intercept for x in range(len(vente_mois))]

# "slope" est la pente de la ligne, représentant le taux de changement des ventes par mois
# "intercept" est l'ordonnée à l'origine, représentant la valeur de départ des ventes
# "r_value" est le coefficient de corrélation, indiquant la force et la direction de la relation linéaire
# "p_value" est la valeur-p, utilisée pour tester l'hypothèse nulle que la pente est nulle
# "std_err" est l'erreur standard de l'estimation de la pente

In [ ]:
plt.figure(figsize=(16, 8))

# Affichage des données
plt.plot(vente_mois.index, vente_mois.values, label='CA par mois', marker='o', linewidth=2)
plt.plot(vente_mois.index, trendline, label='Tendance', color='green', marker='o', linewidth=2)
plt.plot(vente_mois.index, rolling_mean, label='Moyenne roulante', color='red', marker='o', linewidth=2)

# Ajouter les étiquettes de valeurs en dessous des coordonnées de chaque point
for i, value in enumerate(vente_mois.values):
    plt.text(vente_mois.index[i], value - 70, f"{int(value)}", ha='center', va='top', fontsize=10)

# Ajouter une grille
plt.grid(True, linestyle='--', alpha=0.5)

# ajouter les titres des axes
plt.title('Evolution du CA avec Tendance Linéaire')
plt.xlabel('Date')
plt.ylabel('CA')
plt.legend()
plt.show()

In [ ]:
# Résample les données par trimestre
vente_trimestre = vente_mois.resample('Q').sum()

# Calcul de la régression linéaire pour la tendance
slope, intercept, r_value, p_value, std_err = linregress(range(len(vente_trimestre)), vente_trimestre.values)
trendline = [slope * x + intercept for x in range(len(vente_trimestre))]

# Calcul de la moyenne roulante avec une fenêtre de 3
rolling_mean = vente_trimestre.rolling(window=3).mean()

plt.figure(figsize=(16, 8))

# Affichage des données
plt.plot(vente_trimestre.index, vente_trimestre.values, label='CA par trimestre', marker='o', linewidth=2)
plt.plot(vente_trimestre.index, trendline, label='Tendance', color='green', marker='o', linewidth=2)
plt.plot(vente_trimestre.index, rolling_mean, label='Moyenne roulante', color='red', marker='o', linewidth=2)

# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)

# Ajout des titres des axes
plt.title('Evolution du CA avec Tendance Linéaire')
plt.xlabel('Date')
plt.ylabel('CA')

# Création des étiquettes de trimestre
quarter_labels = []
for date in vente_trimestre.index:
    quarter = (date.month - 1) // 3 + 1
    quarter_labels.append(f'Q{quarter} {date.year}')

x_offset = pd.Timedelta(days=20)  # Décalage horizontal en jours
y_offset = 8000  # Décalage vertical

# Ajout des valeurs de chaque point pour le CA par trimestre
for i, value in enumerate(vente_trimestre.values):
    plt.text(vente_trimestre.index[i] + x_offset, value + y_offset, f"{int(value)}", fontsize=9, color='blue')

# Ajout des valeurs de chaque point pour la tendance
for i, value in enumerate(trendline):
    plt.text(vente_trimestre.index[i] + x_offset, value - y_offset, f"{int(value)}", fontsize=9, color='green')

# Ajout des valeurs de chaque point pour la moyenne roulante
for i, value in enumerate(rolling_mean.dropna().values):
    plt.text(rolling_mean.dropna().index[i] + x_offset, value + y_offset, f"{int(value)}", fontsize=9, color='red')
plt.xticks(vente_trimestre.index, quarter_labels)

plt.legend()
plt.show()

##  Zoom sur les réfrences pour voir top et flop

### Références les plus vendues et moins vendues en nombre de ventes

In [ ]:
# Comparer les colonnes 'id_product' des deux DataFrames
# 'isin' renvoie un masque booléen, True pour les valeurs présentes dans 'products' mais absentes dans 'dft'
produit_sans_ventes = products[~products['id_prod'].isin(dft['id_prod'])]

# Afficher les valeurs absentes
print(produit_sans_ventes['id_prod'])

In [ ]:
# Grouper par 'id_prod' et compter le nombre de ventes pour chaque référence
ventes_par_reference = dft['id_prod'].value_counts()

# Tri décroissant pour les références les plus vendues
references_plus_vendues = ventes_par_reference.sort_values(ascending=False)

# Affichage des références les plus vendues (les 10 premières lignes)
print("Références les plus vendues:")
print(references_plus_vendues.head(10))

In [ ]:
# Tracer le graphique en barres horizontales pour les références les plus vendues (top 15)
plt.figure(figsize=(12, 6))

# Trier les valeurs
references_plus_vendues_sorted = references_plus_vendues.head(10).sort_values()

# Utilisation de barh pour les barres horizontales
bar_colors = plt.cm.get_cmap('viridis_r')(range(len(references_plus_vendues_sorted.index)))
plt.barh(references_plus_vendues_sorted.index, references_plus_vendues_sorted.values, color=bar_colors)

# Ajouter des étiquettes de valeur à droite de chaque barre
for i, value in enumerate(references_plus_vendues_sorted.values):
    plt.text(value + 1, i, f"{value}", va='center')

# Personnaliser l'apparence du graphique
plt.xlabel('Nombre de ventes', fontsize=12)
plt.ylabel('Références (id_prod)', fontsize=12)
plt.title('Top 15 des produits les plus vendus', fontsize=14)
plt.xticks(fontsize=10)
plt.tight_layout()

# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)

# Afficher le graphique
plt.show()


### Références les plus vendus en CA

In [ ]:
# Les références les plus vendues (top)
top_references = dft.groupby('id_prod').sum()['price'].nlargest(10)

# Trier les références par ordre décroissant
top_references_sorted = top_references.sort_values(ascending=True)

In [ ]:
plt.figure(figsize=(12, 6))

# Tracer le graphique des références les plus vendues
bar_colors = plt.cm.get_cmap('viridis_r')(range(len(top_references_sorted.index)))
plt.barh(top_references_sorted.index, top_references_sorted.values, color=bar_colors)  # Utilisation de barh pour les barres horizontales

# Ajouter des étiquettes de valeur à droite de chaque barre
for i, value in enumerate(top_references_sorted.values):
    plt.text(value + 1, i, f"{value:.2f}€", va='center')

# Personnaliser l'apparence du graphique
plt.xlabel('Montant total des ventes', fontsize=12)
plt.ylabel('Références', fontsize=12)
plt.title('Références les plus vendues en CA', fontsize=14)
plt.xticks(fontsize=10)
plt.tight_layout()

# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)

# Afficher le graphique
plt.show()

In [ ]:
# Calculer le chiffre d'affaires total
total_CA = dft['price'].sum()

# Calculer le chiffre d'affaires pour les références les plus vendues (top 10)
top_references_CA = top_references.sum()

# Calculer le pourcentage du CA total réalisé par ces produits
top_references_percentage = (top_references_CA / total_CA) * 100

print(f"Les 10 produits les plus vendus en termes de CA représentent {top_references_percentage:.2f}% du chiffre d'affaires total.")

### Références les moins vendues en CA

In [ ]:
# Les références les moins vendues (flop)
flop_references = dft.groupby('id_prod').sum()['price'].nsmallest(10)

# Trier les références par ordre de grandeur (du plus petit au plus grand)
flop_references_sorted = flop_references.sort_values()

In [ ]:
plt.figure(figsize=(12, 6))

# Tracer le graphique des références les moins vendues
bar_colors = plt.cm.get_cmap('viridis_r')(range(len(flop_references_sorted.index)))
plt.barh(flop_references_sorted.index, flop_references_sorted.values, color=bar_colors)

# Ajouter des étiquettes de valeur à droite de chaque barre (collées aux colonnes)
for i, value in enumerate(flop_references_sorted.values):
    plt.text(value, i, f"{value:.2f}€", va='center', ha='left')  # Utiliser ha='left' pour l'alignement horizontal à gauche

# Personnaliser l'apparence du graphique
plt.xlabel('Montant total des ventes', fontsize=12)
plt.ylabel('Références', fontsize=12)
plt.title('Références les moins vendues en CA', fontsize=14)
plt.xticks(fontsize=10)
plt.tight_layout()
# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)
# Afficher le graphique
plt.show()

In [ ]:
dft.head()

## Répartition du CA par catégorie

In [ ]:
# Répartition par catégorie
category_sales = dft.groupby('categ').sum()['price']
total_sales = category_sales.sum()

# Calculer la répartition en pourcentage
category_sales_percentage = (category_sales / total_sales) * 100

# Trier les catégories par ordre de grandeur
category_sales_percentage_sorted = category_sales_percentage.sort_values(ascending=True)

In [ ]:
plt.figure(figsize=(12, 6))

# Tracer le graphique de la répartition par catégorie en pourcentage
plt.barh(category_sales_percentage_sorted.index, category_sales_percentage_sorted.values)  # Utilisation de barh pour les barres horizontales
plt.xlabel('Pourcentage du chiffre d\'affaires total')
plt.ylabel('Catégorie')
plt.title('Répartition du chiffre d\'affaires par catégorie (en pourcentage du total)')

# Couleurs pour chaque catégorie
colors = ['skyblue', 'orange', 'green']
# Tracer le graphique de la répartition par catégorie en pourcentage
plt.barh(category_sales_percentage_sorted.index, category_sales_percentage_sorted.values, color=colors)  # Utilisation de barh pour les barres horizontales

# Ajouter des étiquettes de pourcentage à droite de chaque barre
for i, value in enumerate(category_sales_percentage_sorted.values):
    plt.text(value, i, f"{value:.2f}%", ha='left', va='center')
# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)
# Afficher le graphique
plt.show()

On remarque que 76% de notre CA vient des catégories 0 et 1.

## info sur les profils de nos clients

### Etude sur la répartition de nos clients en focntion du sexe

In [ ]:
# Compter le nombre de clients dans chaque sexe
age_counts = dft['sex'].value_counts()

In [ ]:
# Calculez les pourcentages correspondant à chaque sexe
total_clients = age_counts.sum()
sex_percentages = (age_counts / total_clients) * 100

# Créer le graphique en barres
plt.figure(figsize=(10, 6))
bars = plt.bar(age_counts.index, age_counts.values, color='skyblue')

# Ajoutez les pourcentages sur les barres
for bar, percentage in zip(bars, sex_percentages):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{percentage:.1f}%', ha='center', va='bottom')

plt.xlabel('Sexe')
plt.ylabel('Nombre de clients')
plt.title('Répartition des clients par sexe avec pourcentages')
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

On remarque que le nombre de femmes et d'hommes dans nos client est relativeemtn égal.

### Etude sur la répartition de nos clients en focntion de la tranche d'age

In [ ]:
# Compter le nombre de clients dans chaque tranche d'âge
age_counts = dft['tranche'].value_counts()

# Trier les tranches d'âge par ordre croissant pour le graphique
age_counts = age_counts.sort_index()

In [ ]:
# Normaliser les valeurs pour la mise à l'échelle des couleurs
norm = plt.Normalize(age_counts.values.min(), age_counts.values.max())

# Choisir une carte de couleurs
cmap = plt.cm.viridis_r

# Créer le graphique en barres
plt.figure(figsize=(12, 6))
bars = plt.bar(age_counts.index, age_counts.values, color=[cmap(norm(value)) for value in age_counts.values])

plt.xlabel('Tranche d\'âge')
plt.ylabel('Nombre de clients')
plt.title('Répartition des clients par tranche d\'âge')
plt.xticks(rotation=45, ha='right')

# Ajouter les valeurs au-dessus des barres
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, int(yval), ha='center', va='bottom')

# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)

# Ajuster les marges pour assurer que les étiquettes s'ajustent bien
plt.tight_layout()

# Afficher le graphique
plt.show()

Maintenant que l'on sait que la majorité de nos clients ont entre 31 et 60 ans, avce une plus forte concentration dans la fourchette 41-50 regardont la répartition par porfil.

### Répartition des clients par âge

In [ ]:
# Compter le nombre de clients dans chaque âge
age_counts = dft['age'].value_counts()

# Trier les âges par ordre croissant pour le graphique
age_counts = age_counts.sort_index()

In [ ]:
# Normaliser les valeurs pour la mise à l'échelle des couleurs
norm = plt.Normalize(age_counts.values.min(), age_counts.values.max())

# Choisir une carte de couleurs
cmap = plt.cm.viridis_r

# Créer le graphique en barres
plt.figure(figsize=(14, 6))

# Créer chaque barre individuellement et définir sa couleur en fonction de sa hauteur
for age, count in age_counts.iteritems():
    plt.bar(age, count, color=cmap(norm(count)))

plt.xlabel('Âge')
plt.ylabel('Nombre de clients')
plt.title('Répartition des clients par âge')
plt.xticks(rotation=45, ha='right') # Ajuster l'angle de rotation et l'alignement

# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)

# Afficher le graphique
plt.show()

La majorité de nos clients ont entre 32 et 55 ans.

### Etude sur la répartition de nos clients en focntion du profil

In [ ]:
# Compter le nombre de clients dans chaque profil
age_counts = dft['profil'].value_counts()

In [ ]:
# Créer une palette de couleurs
colors = sns.color_palette('viridis', n_colors=len(age_counts))

# Créer le graphique en barres
plt.figure(figsize=(14, 6)) # Vous pouvez ajuster la taille du graphique
bars = plt.bar(age_counts.index, age_counts.values, color=colors)
plt.xlabel('Profil')
plt.ylabel('Nombre de clients')
plt.title('Répartition des clients par profil')
plt.xticks(rotation=90, ha='right') # Ajuster l'angle de rotation et l'alignement

# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)

# Ajouter les valeurs au-dessus des barres
for bar, color in zip(bars, colors):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, int(yval), ha='center', va='bottom', color=color)

# Ajuster les marges pour assurer que les étiquettes s'ajustent bien
plt.tight_layout()

# Afficher le graphique
plt.show()


On voit que le profil le plus réprésenté est celui des hommes entre 41 et 50 ans, Suivi pas les femmes du même âge. Regardons maintenant la répartition hommes femmes de notre populayion

## Répartition du CA par clients

### Calcul du CA par profil

In [ ]:
# Calcul du chiffre d'affaires par profil client
profit_vente = dft.groupby('profil')['price'].sum()
total_vente = profit_vente.sum()

# Calcul de la répartition en pourcentage
profit_vente_pourcentage = (profit_vente / total_vente) * 100

In [ ]:
# Trier le DataFrame dans l'ordre décroissant
profit_vente_pourcentage_trie = profit_vente_pourcentage.sort_values(ascending=False)

# Choisir une carte de couleurs
cmap = plt.cm.viridis_r

# Normaliser les valeurs pour qu'elles se situent entre 0 et 1
norm = plt.Normalize(vmin=profit_vente_pourcentage_trie.min(), vmax=profit_vente_pourcentage_trie.max())

# Affichage du diagramme à barres trié en pourcentage
plt.figure(figsize=(14, 6))
bars = plt.bar(profit_vente_pourcentage_trie.index, profit_vente_pourcentage_trie.values, color=[cmap(norm(value)) for value in profit_vente_pourcentage_trie.values])
plt.xlabel('Profil client')
plt.ylabel('Pourcentage du chiffre d\'affaires total')
plt.title('Répartition du chiffre d\'affaires par profil client (trié décroissant, en pourcentage du total)')
plt.xticks(rotation=45, ha='right')

# Ajouter des étiquettes de pourcentage au-dessus de chaque barre
for bar, value in zip(bars, profit_vente_pourcentage_trie.values):
    plt.annotate(f"{value:.2f}%", (bar.get_x() + bar.get_width() / 2, value), ha='center', va='bottom')

# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)

# Ajuster les marges pour assurer que les étiquettes s'ajustent bien
plt.tight_layout()

# Afficher le graphique
plt.show()


## Courbe de lorenz

La courbe de Lorenz est tracée en utilisant deux axes : l'axe horizontal représente le pourcentage cumulé des individus dans la population, tandis que l'axe vertical représente le pourcentage cumulé de la variable mesurée (par exemple, le chiffre d'affaires, le revenu, etc.).

In [ ]:
dft.head()

In [ ]:
# Calculer le chiffre d'affaires total par profil client
ca_profil = dft.groupby('profil')['price'].sum()

# Calculer le pourcentage de chiffre d'affaires réalisé par chaque profil client
profit_vente_pourcentage = (ca_profil / ca_profil.sum()) * 100

# Trier les profils clients en fonction du chiffre d'affaires
profil_trie = profit_vente_pourcentage.sort_values(ascending=True) # changer le tri ?

# Calculer les pourcentages cumulés du chiffre d'affaires
pourcentage_cumule_ca = np.cumsum(profil_trie)

# Calculer les pourcentages cumulés de la population
population_cumule = np.arange(1, len(profil_trie) + 1) / len(profil_trie)

In [ ]:
# Tracer la courbe de Lorenz avec la grille et les valeurs d'en-têtes
plt.figure(figsize=(14, 6))

# Tracer la courbe de Lorenz
plt.plot(population_cumule, pourcentage_cumule_ca, label='Courbe de Lorenz')

# Ajouter les points sur la courbe de Lorenz
plt.scatter(population_cumule, pourcentage_cumule_ca, color='blue')

plt.plot([0, 1], [0, 100], color='red', linestyle='--', label='Égalité parfaite')

plt.xlabel('Pourcentage de population')
plt.ylabel('Pourcentage du chiffre d\'affaires')
plt.title('Courbe de Lorenz du chiffre d\'affaires par profil client')
plt.legend()
# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(np.arange(0, 1.1, 0.1))  # Définir les étiquettes de l'axe x
plt.yticks(np.arange(0, 101, 10))  # Définir les étiquettes de l'axe y

plt.show()

On remarque que 50% de notre population représente 70% de notre CA.

In [ ]:
# Calculer l'aire sous la courbe de Lorenz en utilisant la méthode des trapèzes
aire_courbe_de_lorenz = np.trapz(pourcentage_cumule_ca, population_cumule).round()

# Afficher l'aire sous la courbe de Lorenz
print("Aire sous la courbe de Lorenz:", aire_courbe_de_lorenz)

L'aire sous la courbe de Lorenz représente la proportion de la totalité de la distribution (dans ce cas, le CA) détenue par une proportion donnée de la population (les clients). Une aire de 37.0 indique que la distribution est fortement inégale, avec une grande proportion du CA provenant d'une petite proportion de la population.

In [ ]:
# Calculer l'aire totale sous la ligne d'égalité parfaite (A)
A = 0.5

# Calculer l'aire entre la courbe de Lorenz et la ligne d'égalité parfaite (B)
B = aire_courbe_de_lorenz - A

# Calculer l'indice de Gini (G)
gini_index = B / A

# Afficher l'indice de Gini
print("Indice de Gini:", gini_index)

Indice de Gini: 73.0
L'indice de Gini est une mesure de l'inégalité qui varie entre 0 et 100. Un indice de 0 représente une distribution parfaitement équitable (chaque client contribue de manière égale au CA), tandis qu'un indice de 100 représente une inégalité totale (tout le CA provient d'un seul client).

Un indice de Gini de 73.0 est considéré comme très élevé et indique une inégalité importante dans la distribution du CA parmi vos clients. Cela signifie que la majorité du CA est générée par une petite proportion de vos clients.



Conclusion
Ces valeurs indiquent que votre entreprise dépend fortement d'un petit segment de clients pour générer la majorité de son chiffre d'affaires

<a id="section5"></a>
# **CORRELATIONS**
---

## le lien entre le genre d'un client et les catégories de livres achetées

In [ ]:
# Grouper les données par 'sex' et 'categ', puis compter le nombre de produits dans chaque groupe
nombre_par_categorie = dft.groupby(['sex', 'categ']).count()['id_prod']

In [ ]:
sexes =['f', 'm']
categories = ['0','1','2']

# Couleurs pour les hommes et les femmes
colors = ['blue', 'yellow']

# Configuration du graphique
x = range(len(categories))
width = 0.35

# Création du graphique à barres
fig, ax = plt.subplots()
for i, sex in enumerate(sexes):
    ax.bar([val + i * width for val in x], nombre_par_categorie[i::len(sexes)], width, label=sex, color=colors[i])

# Paramètres du graphique
ax.set_xlabel('Catégorie')
ax.set_ylabel('Nombre de produits')
ax.set_title('Nombre de produits par sexe et catégorie')
ax.set_xticks([val + width / 2 for val in x])
ax.set_xticklabels(categories)
ax.legend()
# Ajout d'une grille
plt.grid(True, linestyle='--', alpha=0.5)
# Affichage du graphique
plt.show()

On remarque que selon les catégories le nombre de produtis acheté par les clietns selon leur sexe varie enormément. On peut penser que la catégorie 1 est plus préféré cehz les hommes et l'inverse pour la catégorie 2. Pour la Catégorie 0 on a une forte présence des hommes mais aussi une forte présence des femmes mais moindre.

On choisit d'utiliser le test du Chi-carré car nous avons deux variables catégorielles. on ne peut donc pas utiliser une corrélation classiques.

In [ ]:
# Création d'un tableau de contingence
contingency_table = pd.crosstab(dft['sex'], dft['categ'])

# Application du test du Chi-carré
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Afficher les résultats
print('Tableau de contingence :')
print(contingency_table)
print()
print('Résultats du test du Chi-carré :')
print()
print('Valeur de Chi2 :', chi2)
print()
print('P-valeur :', p)
print()
print('Degrés de liberté :', dof)
print()
print('Tableau des fréquences attendues :')
expected_table = pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns)
expected_table = expected_table.round() 
print(expected_table)

In [ ]:
# Calcul de la différence entre les deux tables
difference_table = contingency_table - expected_table

# Calcul du pourcentage de la différnce
percentage_difference_table = round((difference_table / expected_table) * 100)

print("\nPercentage Difference Table:")

# Creation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(percentage_difference_table, annot=True, cmap='coolwarm')
plt.show()

Ces résultats indiquent que le test du Chi-carré a trouvé une association statistiquement significative entre le sexe du client et la catégorie de produits qu'ils achètent.

La valeur de Chi2 est assez élevée, ce qui indique une forte déviation par rapport à ce que nous attendrions si les deux variables étaient indépendantes. De plus, la p-valeur est extrêmement petite (bien en dessous du seuil standard de 0,05), ce qui signifie que le résultat est très probablement statistiquement significatif et non dû au hasard.

En conclusion, sur la base de ces résultats, il semble qu'il existe une corrélation entre le sexe d'un client et la catégorie de produits qu'il achète. Cependant, cela ne signifie pas nécessairement qu'il y a une relation de cause à effet. D'autres facteurs pourraient être à l'œuvre. (age, revenu,...)

## le lien entre l'âge des clients et le montant total des achats

In [ ]:
# Grouper les données par âge et calculer la somme du montant total des achats
age_total_purchase = dft.groupby('age')['price'].sum()

In [ ]:
# Obtenir les valeurs d'âge et les montants totaux des achats
ages = age_total_purchase.index
purchase_totals = age_total_purchase.values

# Normaliser les valeurs d'achat pour la mise à l'échelle des couleurs
norm = plt.Normalize(purchase_totals.min(), purchase_totals.max())

# Choisir une carte de couleurs
cmap = plt.cm.viridis_r

# Créer le graphique à barres
plt.figure(figsize=(14, 6))
plt.bar(ages, purchase_totals, color=[cmap(norm(value)) for value in purchase_totals])
plt.xlabel('Âge')
plt.ylabel('Montant total des achats')
plt.title('Répartition du montant total des achats en fonction de l\'âge')

# Ajouter une grille
plt.grid(True, linestyle='--', alpha=0.5)

# Afficher le graphique
plt.show()

On remarque que nous avons beaucoup de personnes de 43 ans qui effectue des achats, ce qui correspond à notre tranche d'age la plus active. Cependant on remarque aussi que beaucoup de personnes de 18 ans effectue des achats. Cela est plus surprenant mais peut s'expliquer par le fait de posséder une carte bleu ou d'être majeur poiur ouvrir un compte client. Donc les mineurs doivent renseigner 18 ans pour pouvoir acheter un produit.

Nosu allons utiliser le coefficient de corrélation de Pearson qui est le plus adpaté pour deux variables continues. il mesure la force et la direction de la relation linéaire entre les variables.

In [ ]:
# Calculer la corrélation et la p-valeur du montant total des achats en fonction de l'age
corr, p_value = pearsonr(dft['age'], dft['price'])
print('Correlation: ', corr)
print()
print('P-value: ', p_value)

In [ ]:
# corrélation entre l'age et le prix 
corr = dft[['age','price']].corr()

# Creation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

La corrélation entre l'âge des clients et le montant total des achats est de -0.20749608778813317, ce qui indique une corrélation négative faible. Le p-value est de 0.0, ce qui suggère que cette corrélation est statistiquement significative. Cela signifie qu'il existe une relation significative entre l'âge des clients et le montant total des achats, mais la corrélation est faible.

Veuillez noter que cette corrélation ne capture qu'une relation linéaire entre les deux variables et d'autres facteurs peuvent également influencer le montant total des achats. Il peut être utile d'explorer d'autres techniques d'analyse pour mieux comprendre la relation entre ces variables.

## le lien entre l'age des clients et la fréquence d'achat

In [ ]:
# Grouper les données par âge et compter le nombre d'achats dans chaque groupe
achat_par_age = dft.groupby('age').count()['id_prod']

In [ ]:
# Normaliser les valeurs de fréquence d'achat pour la mise à l'échelle des couleurs
norm = plt.Normalize(achat_par_age.values.min(), achat_par_age.values.max())

# Choisir une carte de couleurs
cmap = plt.cm.viridis_r

# Créer le graphique à barres
fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(achat_par_age.index, achat_par_age.values, color=[cmap(norm(value)) for value in achat_par_age.values])

# Paramètres du graphique
ax.set_xlabel('Âge')
ax.set_ylabel('Fréquence d\'achat')
ax.set_title('Fréquence d\'achat en fonction de l\'âge')

# Ajouter une grille
ax.grid(True, linestyle='--', alpha=0.5)

# Affichage du graphique
plt.show()

On retrouve les personnes de 43 ans qui sont les plus présentes dans nos statistique en termes d'aceht et de présence. De 31 à 55 est la tranche d'age la plus active en terle de fréquence d'achat.

Nosu allons utiliser le coefficient de corrélation de Pearson qui est le plus adpaté pour deux variables continues. il mesure la force et la direction de la relation linéaire entre les variables.

In [ ]:
# Calculer la corrélation et la p-valeur de la fréquence d'achat en fonction de l'age
corr, p_value = pearsonr(dft['age'], dft['frequence_achat'])
print('Correlation: ', corr)
print()
print('P-value: ', p_value)

In [ ]:
# corrélation avec la fonction pandas
corr = dft[['age','frequence_achat']].corr()

# Creation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

La corrélation de Pearson entre l'âge des clients et la fréquence d'achat est de -0.0185, ce qui indique une faible corrélation entre l'âge et la fréquence d'acaht des cleitns. La p-valeur associée est très faible (4.09e-53), ce qui suggère une corrélation significative entre l'age et la fréquence d'achat.

## le lien entre l'age du client et la taille du panier moyen

### Par tranche d'ages

In [ ]:
# Grouper les données par âge et calculer la taille moyenne du panier
panier_moyen_par_tranche = dft.groupby('tranche')['panier_moyen_par_age'].mean()

In [ ]:
# Obtenir les valeurs minimales et maximales
min_value = panier_moyen_par_tranche.min()
max_value = panier_moyen_par_tranche.max()

# Créer un diagramme à barres
plt.figure(figsize=(14, 6))
bars = plt.bar(panier_moyen_par_tranche.index, panier_moyen_par_tranche.values, alpha=0.5)

# Appliquer la couleur en fonction de la hauteur des barres
for bar, value in zip(bars, panier_moyen_par_tranche.values):
    bar.set_color(plt.cm.viridis_r((value - min_value) / (max_value - min_value)))

plt.xlabel('Âge')
plt.ylabel('Taille du panier moyen en euros')
plt.title('Lien entre la tranche d\'age du client et la taille du panier moyen')

plt.show()

### Par age

In [ ]:
# Grouper les données par âge et calculer la taille moyenne du panier
panier_moyen_par_age = dft.groupby('age')['panier_moyen_par_age'].mean()

In [ ]:
dft['panier_moyen_par_age'].mean()

In [ ]:
# Obtenir les valeurs minimales et maximales
min_value = panier_moyen_par_age.min()
max_value = panier_moyen_par_age.max()

# Créer un diagramme à barres
plt.figure(figsize=(14, 6))
bars = plt.bar(panier_moyen_par_age.index, panier_moyen_par_age.values, alpha=0.5)

# Appliquer la couleur en fonction de la hauteur des barres
for bar, value in zip(bars, panier_moyen_par_age.values):
    bar.set_color(plt.cm.viridis_r((value - min_value) / (max_value - min_value)))

plt.xlabel('Âge')
plt.ylabel('Taille du panier moyen en euros')
plt.title('Lien entre l\'âge du client et la taille du panier moyen')

plt.show()

On remarque que les personnes entre 18 et 31 ans ont un panier moyen largement supérieur au reste de notre population.

### Corrélations

Nosu allons utiliser le coefficient de corrélation de Pearson qui est le plus adpaté pour deux variables continues. il mesure la force et la direction de la relation linéaire entre les variables.

In [ ]:
# Calculer la corrélation et la p-valeur entre la taille du panier moyen et l'age
corr, p_value = pearsonr(dft['age'], dft['panier_moyen_par_age'])
print('Correlation: ', corr)
print()
print('P-value: ', p_value)

In [ ]:
# calcul corrélation
corr = dft[['age','panier_moyen_par_age']].corr()

# Creation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

La corrélation entre l'âge du client et la taille du panier moyen est de -0.4326, la corrélation est modérée, ce qui suggère qu'il existe une tendance à une diminution de la taille du panier moyen avec l'augmentation de l'âge. La p-valeur est de 0.0, ce qui suggère que cette corrélation est statistiquement significative. Cela signifie qu'il existe une relation significative entre l'âge du client et la taille de son panier moyen.

## le lien entre l'age et la catégorie de livre achetés

2 approches au problème, une avec els tranches d'ages et l'autre avec l'age.

### Lien entre la tranche d'age et la catégorie de livre achetés

#### Répartition du nombre de produits de la catégorie 0 en fonction de la tranche d'age

In [ ]:
# Filtrer les données pour la catégorie 0
category_0 = dft[dft['categ'] == '0']

# Grouper les données par âge et compter le nombre de produits dans chaque groupe
category_0_counts = category_0.groupby('age')['id_prod'].count()

# Grouper les données par tranche d'âge et compter le nombre de produits dans chaque groupe
category_counts_tranche_0 = category_0.groupby('tranche')['id_prod'].count()
category_counts_tranche_0

In [ ]:
# Données pour les catégories d'âge et le nombre de produits
tranche = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '70+']
nombre_produits = [0, 14412, 119972, 177018, 62257, 24801, 16999]

# Normaliser les valeurs de nombre_produits
norm = plt.Normalize(min(nombre_produits), max(nombre_produits))

# Créer une palette de couleurs basée sur la normalisation
colors = [plt.cm.viridis_r(norm(value)) for value in nombre_produits]

# Créer le graphique à barres
plt.figure(figsize=(14, 6))
bars = plt.bar(tranche, nombre_produits, color=colors)

# Ajouter des étiquettes et des titres
plt.xlabel('Tranche d\'âge')
plt.ylabel('Nombre de produits')
plt.title('Nombre de produits de la catégorie 0 par tranche d\'âge')

# Afficher le graphique
plt.show()

Les 31-50 ans sont ceux qui achète le plus de la catégorie 0

#### Répartition du nombre de produits de la catégorie 1 en fonction de la tranche d'age

In [ ]:
# Filtrer les données pour la catégorie 1
category_1 = dft[dft['categ'] == '1']

# Grouper les données par âge et compter le nombre de produits dans chaque groupe
category_1_counts = category_1.groupby('age')['id_prod'].count()

# Grouper les données par tranche d'âge et compter le nombre de produits dans chaque groupe
category_counts_tranche_1 = category_1.groupby('tranche')['id_prod'].count()
category_counts_tranche_1

In [ ]:
# Données pour les catégories d'âge et le nombre de produits
tranche = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '70+']
nombre_produits = [0, 27656, 41628, 56309, 54896, 32685, 22418]

# Normaliser les valeurs de nombre_produits
norm = plt.Normalize(min(nombre_produits), max(nombre_produits))

# Créer une palette de couleurs basée sur la normalisation
colors = [plt.cm.viridis_r(norm(value)) for value in nombre_produits]
# Créer le graphique à barres
plt.figure(figsize=(14, 6))
plt.bar(tranche, nombre_produits, color=colors)

# Ajouter des étiquettes et des titres
plt.xlabel('Tranche d\'âge')
plt.ylabel('Nombre de produits')
plt.title('Nombre de produits de la categorie 1 par tranche d\'âge')

# Afficher le graphique
plt.show()

Pour la catégorie 1 la répartition est plus équilibré entre les tranches d'ages malgré une plius importante présence des 41-60 ans dnas notre population.

#### Répartition du nombre de produits de la catégorie 2 en fonction de la tranche d'age

In [ ]:
# Filtrer les données pour la catégorie 2
category_2 = dft[dft['categ'] == '2']

# Grouper les données par âge et compter le nombre de produits dans chaque groupe
category_2_counts = category_2.groupby('age')['id_prod'].count()

# Grouper les données par tranche d'âge et compter le nombre de produits dans chaque groupe
category_counts_tranche_2 = category_2.groupby('tranche')['id_prod'].count()
category_counts_tranche_2

In [ ]:
# Données pour les catégories d'âge et le nombre de produits
tranche = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '70+']
nombre_produits = [0, 29063, 4696, 769, 1056, 550, 349]

# Normaliser les valeurs de nombre_produits
norm = plt.Normalize(min(nombre_produits), max(nombre_produits))

# Créer une palette de couleurs basée sur la normalisation
colors = [plt.cm.viridis_r(norm(value)) for value in nombre_produits]

# Créer le graphique à barres
plt.figure(figsize=(14, 6))
plt.bar(tranche, nombre_produits, color=colors)

# Ajouter des étiquettes et des titres
plt.xlabel('Catégorie d\'âge')
plt.ylabel('Nombre de produits')
plt.title('Nombre de produits de la categorie 2 par tranche d\'âge')

# Afficher le graphique
plt.show()

Pour la catégorie 2 les 19-30 sont largement majoritaire.

### Lien entre l'age et la catégorie de livre achetés

#### Répartition du nombre de produits de la catégorie 0 en fonction de l'age

In [ ]:
# Normaliser les valeurs de category_0_counts
norm = plt.Normalize(category_0_counts.min(), category_0_counts.max())

# Créer une palette de couleurs basée sur la normalisation
colors = [plt.cm.viridis_r(norm(value)) for value in category_0_counts]

# Créer les positions des âges sur l'axe x
x = np.arange(len(category_0_counts))

# Créer la figure et l'axe avec une taille agrandie
plt.figure(figsize=(14, 8))
ax = plt.gca()

# Créer les barres
ax.bar(x, category_0_counts, width=0.5, color=colors)

# Définir les étiquettes de l'axe x et le titre du graphique
ax.set_xticks(x)
ax.set_xticklabels(category_0_counts.index, rotation=45, ha='right')
ax.set_xlabel('Âge')
ax.set_ylabel('Nombre de produits')
ax.set_title('Répartition du nombre de produits de la catégorie 0 en fonction de l\'âge')

# Ajuster les marges pour éviter que les étiquettes ne soient coupées
plt.tight_layout()

# Afficher le graphique
plt.show()

#### Répartition du nombre de produits de la catégorie 1 en fonction de l'age

In [ ]:
# Normaliser les valeurs de category_1_counts
norm = plt.Normalize(category_1_counts.min(), category_1_counts.max())

# Créer une palette de couleurs basée sur la normalisation
colors = [plt.cm.viridis_r(norm(value)) for value in category_1_counts]

# Créer les positions des âges sur l'axe x
x = np.arange(len(category_1_counts))

# Créer la figure et l'axe avec une taille agrandie
plt.figure(figsize=(14, 8))
ax = plt.gca()

# Créer les barres
ax.bar(x, category_1_counts, width=0.5, color=colors)

# Définir les étiquettes de l'axe x et le titre du graphique
ax.set_xticks(x)
ax.set_xticklabels(category_1_counts.index, rotation=45, ha='right')
ax.set_xlabel('Âge')
ax.set_ylabel('Nombre de produits')
ax.set_title('Répartition du nombre de produits de la catégorie 1 en fonction de l\'âge')

# Ajuster les marges pour éviter que les étiquettes ne soient coupées
plt.tight_layout()

# Afficher le graphique
plt.show()

#### Répartition du nombre de produits de la catégorie 2 en fonction de l'age

In [ ]:
# Normaliser les valeurs de category_2_counts
norm = plt.Normalize(category_2_counts.min(), category_2_counts.max())

# Créer une palette de couleurs basée sur la normalisation
colors = [plt.cm.viridis_r(norm(value)) for value in category_2_counts]

# Créer les positions des âges sur l'axe x
x = np.arange(len(category_2_counts))

# Créer la figure et l'axe avec une taille agrandie
plt.figure(figsize=(14, 8))
ax = plt.gca()

# Créer les barres
ax.bar(x, category_2_counts, width=0.5, color=colors)

# Définir les étiquettes de l'axe x et le titre du graphique
ax.set_xticks(x)
ax.set_xticklabels(category_2_counts.index, rotation=45, ha='right')
ax.set_xlabel('Âge')
ax.set_ylabel('Nombre de produits')
ax.set_title('Répartition du nombre de produits de la catégorie 2 en fonction de l\'âge')

# Ajuster les marges pour éviter que les étiquettes ne soient coupées
plt.tight_layout()

# Afficher le graphique
plt.show()

### Test ANOVA

In [ ]:
#  ANOVA test
anova_result = f_oneway(*[dft[dft['age'] == tr]['categ'] for tr in dft['age'].unique()])

# Afficher les résultats
print("ANOVA Test Results:")
print("F-statistic:", anova_result.statistic)
print("P-value:", anova_result.pvalue)

la F-statistic est de 3489.842 et la P-value est de 0.0. Cela signifie qu'il existe des différences significatives entre les moyennes des groupes analysés et que ces différences ne sont pas dues au hasard. La P-value étant nulle, cela indique une très forte confiance statistique dans le rejet de l'hypothèse nulle selon laquelle les moyennes des groupes seraient égales. (l'hypothèse nulle est que les moyennes de tous les groupes sont égales) .

### Test du Chi-carré

Pour calculer la corrélation entre la tranche d'age et la categ on va utilsier le test du chi-carré, car ce sont 2 vairables catégorielles.

In [ ]:
# Créer le tableau de contingence entre 'tranchr' et 'categ'
contingency_table = pd.crosstab(dft['tranche'], dft['categ'])

# Appliquer le test du Chi-carré
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Afficher les résultats
print('Tableau de contingence :')
print(contingency_table)
print()
print('Résultats du test du Chi-carré :')
print('Valeur de Chi2 :', chi2)
print('P-valeur :', p_value)
print('Degrés de liberté :', dof)
print()
print('Tableau des fréquences attendues :')
expected_table = pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns)
expected_table = expected_table.round()
print(expected_table)

In [ ]:
# Calcul de la différence entre les deux tables
difference_table = contingency_table - expected_table

# Calcul du pourcentage de la différnce
percentage_difference_table = round((difference_table / expected_table) * 100)

print("\nPercentage Difference Table:")
print(percentage_difference_table)

# Creation heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(percentage_difference_table, annot=True, cmap='coolwarm')
plt.show()

Les résultats du test du Chi-carré indiquent une association statistiquement significative entre les variables "age" et "categ".

La valeur de Chi2 est élevée (305629.37802685937), ce qui suggère une forte déviation par rapport à ce que nous attendrions si les deux variables étaient indépendantes. La p-valeur est de zéro 0.0, ce qui indique que le résultat n'est pas du au hasard.

Le tableau de contingence montre le nombre d'observations pour chaque combinaison d'âge et de catégorie. Le tableau des fréquences attendues présente les fréquences théoriques attendues sous l'hypothèse nulle d'indépendance entre les variables.

Ces résultats suggèrent donc qu'il existe une relation significative entre l'âge et la catégorie de produits achetés. Cependant, il est important de noter que le test du Chi-carré mesure uniquement l'association entre les variables et ne fournit pas d'indication sur la direction ou la force de la relation. Pour obtenir des informations plus détaillées sur la nature de cette relation, il peut être utile d'utiliser d'autres mesures de corrélation adaptées aux variables catégorielles.

<a id="section6"></a>
# **CONCLUSION**
---

**Aperçu du Profil Client :**

Nos clients sont segmentés selon divers critères, dont l'âge et le sexe. L'analyse dévoile que la tranche d'âge dominante se situe entre 31 et 60 ans, avec une prépondérance pour les individus âgés de 41 à 50 ans. La proportion de clients masculins et féminins est pratiquement équilibrée.

Il est intéressant de noter une forte activité d'achat chez les individus de 43 ans. Par ailleurs, un nombre significatif d'achats provient d'individus de 18 ans. Ceci pourrait être attribué à l'obtention d'une carte bancaire ou à l'âge légal pour créer un compte client, poussant ainsi certains mineurs à indiquer avoir 18 ans.

**Dynamique d'Achat :**

En explorant les tendances d'achat, on constate que la majorité de notre chiffre d'affaires est générée par les catégories 0 et 1. Les préférences d'achat varient notablement en fonction du sexe : la catégorie 1 semble plus prisée par les hommes, tandis que la catégorie 2 semble avoir la faveur des femmes. La catégorie 0, quant à elle, attire à la fois hommes et femmes, bien que ces dernières soient légèrement moins représentées.

**Interconnexions Notables :**

Le sexe du client influence la catégorie de produits qu'il privilégie.
L'âge impacte le volume d'achats, bien que la corrélation soit modeste.
Une corrélation légère existe entre l'âge et la régularité des achats.
L'âge influence également le montant moyen du panier d'achat.
Il existe un lien tangible entre l'âge du client et la catégorie de produits qu'il choisit.

**Examen des Catégories de Produits :**

Cette étude a révélé des tendances d'achat claires parmi nos clients. Par exemple, les jeunes adultes (18-31 ans) tendent à avoir un panier d'achat supérieur à la moyenne. Les clients âgés de 43 ans se démarquent par leur activité d'achat, et la tranche d'âge 31-55 ans est la plus assidue en matière d'achats.

**Perspectives Financières :**

En évaluant la distribution du chiffre d'affaires, nous constatons que les 10 produits les plus vendus génèrent plus de 5% de notre chiffre d'affaires total. De plus, une moitié significative de notre clientèle contribue à près de 70% de notre chiffre d'affaires, soulignant l'importance de fidéliser ce segment.

<a id="section6"></a>
# **AJOUT suite à la soutenance**
---

In [ ]:
from scipy.stats import shapiro
from scipy.stats import spearmanr
from scipy import stats

1. **Test de corrélation de Pearson**:
    - Age et montant
    - Age et fréquence
    - Age et montant total
2. **Vérification de la normalité des distributions**:
    - Pour l'âge, nous utiliserons le test de Shapiro-Wilk et le test de Kolmogorov-Smirnov.
3. **Si l'âge ne présente pas une distribution normale**:
    - Nous utiliserons le test de corrélation de Spearman à la place de Pearson pour les associations mentionnées précédemment.
4. **Pour l'ANOVA**:
    - Test de Shapiro-Wilk pour vérifier la normalité de l'âge au sein de chaque catégorie.
    - Test de Levene pour vérifier l'homogénéité des variances.
5. **Si les conditions pour l'ANOVA ne sont pas remplies**:
    - Nous utiliserons le test de Kruskal-Wallis pour la comparaison des médianes.



In [644]:
dft.head()

,id_prod,date,session_id,client_id,price,categ,sex,birth,age,tranche,profil,panier_moyen_par_age,panier_moyen_par_tranche_age,frequence_achat
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,11.99,0,f,1967,56,51-60,Femme entre 50-60 ans,17.0,16.0,63
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,19.37,0,m,1960,63,61-70,Homme entre 60-70 ans,17.0,17.0,117
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,4.50,0,m,1988,35,31-40,Homme entre 30-40 ans,13.0,15.0,338
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,6.55,0,f,1989,34,31-40,Femme entre 30-40 ans,13.0,15.0,239
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,16.49,0,f,1956,67,61-70,Femme entre 60-70 ans,17.0,17.0,82


In [ ]:

# Test de normalité pour la variable 'age'
stat, p = shapiro(dft['age'])
print('Test de Shapiro-Wilk pour l\'âge :')
print(f'Statistique de test : {stat}')
print(f'P-valeur : {p}')
print()

# Interprétation des résultats
alpha = 0.05
if p > alpha:
    print('L\'échantillon semble provenir d\'une distribution normale (l\'hypothèse nulle n\'est pas rejetée)')
else:
    print('L\'échantillon ne semble pas provenir d\'une distribution normale (l\'hypothèse nulle est rejetée)')

Le test de Shapiro-Wilk indique que la distribution de l'âge dans votre échantillon ne suit pas une distribution normale. La p-valeur de 0.0 est inférieure à un seuil de significativité communément utilisé de 0,05, ce qui signifie que vous pouvez rejeter l'hypothèse nulle selon laquelle les données sont normalement distribuées.

Cela signifie que, d'après ce test, l'âge de vos clients ne suit pas une distribution normale dans votre ensemble de données. 



In [ ]:
# Test de corrélation de Spearman entre l'âge et le montant
rho, p = spearmanr(dft['age'], dft['price'])
print('Test de corrélation de Spearman entre l\'âge et le montant :')
print(f'Coefficient de corrélation de Spearman (rho) : {rho}')
print(f'P-valeur : {p}')

Le test de corrélation de Spearman entre l'âge et le montant indique une corrélation statistiquement significative, bien que faible, entre ces deux variables. Voici ce que signifient les résultats :

Coefficient de corrélation de Spearman (rho) : -0.055464894621283005

Ce coefficient mesure la force et la direction de la relation monotone entre l'âge et le montant. Dans ce cas, un coefficient négatif indique une relation décroissante, c'est-à-dire que, en général, à mesure que l'âge augmente, le montant a tendance à diminuer légèrement.

P-valeur : 0.0

La p-valeur est très faible (proche de zéro), ce qui indique que la corrélation est statistiquement significative. Cela signifie que la corrélation observée n'est probablement pas due au hasard.

Bien que la corrélation soit statistiquement significative, il est important de noter que le coefficient de corrélation de Spearman est très proche de zéro, ce qui suggère une relation faible entre l'âge et le montant. 



In [652]:
# Test de corrélation de Spearman entre l'âge et la fréquence d'achat
rho, p = spearmanr(dft['age'], dft['frequence_achat'])
print('Test de corrélation de Spearman entre l\'âge et la fréquence :')
print(f'Coefficient de corrélation de Spearman (rho) : {rho}')
print(f'P-valeur : {p}')


Test de corrélation de Spearman entre l'âge et la fréquence :
Coefficient de corrélation de Spearman (rho) : -0.039607443298568246
P-valeur : 9.855725903289696e-237


Le test de corrélation de Spearman  entre l'âge et la fréquence d'achat indique une corrélation statistiquement significative, bien que faible, entre ces deux variables. Voici ce que signifient les résultats :

Coefficient de corrélation de Spearman (rho) : -0.039607443298568246

Ce coefficient mesure la force et la direction de la relation monotone entre l'âge et la fréquence d'achat. Dans ce cas, un coefficient négatif indique une relation décroissante, ce qui signifie que, en général, à mesure que l'âge augmente, la fréquence d'achat a tendance à diminuer légèrement.

P-valeur : 9.855725903289696e-237

La p-valeur est extrêmement faible (proche de zéro), ce qui indique que la corrélation est statistiquement significative. Cela signifie que la corrélation observée n'est probablement pas due au hasard.

Tout comme dans le cas de la corrélation entre l'âge et le montant, la corrélation entre l'âge et la fréquence est statistiquement significative mais faible. 


In [654]:
# Grouper les données par âge et calculer la somme du montant total des achats
age_total_purchase = dft.groupby('age')['price'].sum()

# Convertir la série en DataFrame
age_total_purchase_df = age_total_purchase.reset_index()

# Renommer les colonnes pour correspondre à celles de votre DataFrame d'origine
age_total_purchase_df.rename(columns={'age': 'age', 'price': 'montant_total'}, inplace=True)

# Effectuer le test de corrélation de Spearman entre l'âge et le montant total
rho, p = spearmanr(age_total_purchase_df['age'], age_total_purchase_df['montant_total'])
print('Test de corrélation de Spearman entre l\'âge et le montant total des achats :')
print(f'Coefficient de corrélation de Spearman (rho) : {rho}')
print(f'P-valeur : {p}')


Test de corrélation de Spearman entre l'âge et le montant total des achats :
Coefficient de corrélation de Spearman (rho) : -0.8583732057416268
P-valeur : 3.805937971547995e-23


Le test de corrélation de Spearman entre l'âge et le montant total des achats indique une corrélation statistiquement significative et assez forte entre ces deux variables. Voici ce que signifient les résultats :

Coefficient de corrélation de Spearman (rho) : -0.8583732057416268

Ce coefficient mesure la force et la direction de la relation monotone entre l'âge et le montant total des achats. Dans ce cas, un coefficient négatif indique une relation décroissante, c'est-à-dire que, en général, à mesure que l'âge augmente, le montant total des achats a tendance à diminuer de manière significative.

P-valeur : 3.805937971547995e-23

La p-valeur est extrêmement faible (proche de zéro), ce qui indique que la corrélation est statistiquement significative. Cela signifie que la corrélation observée n'est probablement pas due au hasard.

Dans ce cas, la corrélation est assez forte, ce qui suggère qu'il y a une relation significative entre l'âge des clients et le montant total de leurs achats. Plus précisément, à mesure que l'âge des clients augmente, leurs dépenses totales ont tendance à diminuer de manière significative.



In [662]:
# Divisez votre DataFrame en groupes par catégorie
groupes_par_categorie = [dft[dft['categ'] == i]['age'] for i in dft['categ'].unique()]

# Test de normalité de l'âge pour chaque catégorie
shapiro_results = [stats.shapiro(groupe) for groupe in groupes_par_categorie]

# Test d'homogénéité des variances (Test de Levene)
levene_stat, levene_p_value = stats.levene(*groupes_par_categorie)

# Afficher les résultats des tests de normalité
for i, (categ, (stat, p_value)) in enumerate(zip(dft['categ'].unique(), shapiro_results)):
    print(f"Test de Shapiro-Wilk pour la catégorie {categ}:")
    print(f"Statistique de test : {stat}")
    print(f"P-valeur : {p_value}")
    if p_value < 0.05:
        print("L'échantillon ne semble pas provenir d'une distribution normale (l'hypothèse nulle est rejetée)")
    else:
        print("L'échantillon semble provenir d'une distribution normale (l'hypothèse nulle n'est pas rejetée)")
    print()

# Afficher le résultat du test d'homogénéité des variances
print("Test d'homogénéité des variances (Test de Levene) :")
print(f"Statistique de test : {levene_stat}")
print(f"P-valeur : {levene_p_value}")
if levene_p_value < 0.05:
    print("L'homogénéité des variances n'est pas vérifiée (l'hypothèse nulle est rejetée)")
else:
    print("L'homogénéité des variances est vérifiée (l'hypothèse nulle n'est pas rejetée)")

# Si les conditions pour l'ANOVA ne sont pas remplies, effectuez le test de Kruskal-Wallis
if any(p_value < 0.05 for _, p_value in shapiro_results) or levene_p_value < 0.05:
    print("\nConditions pour l'ANOVA non remplies. Effectuez le test de Kruskal-Wallis.")
    kruskal_stat, kruskal_p_value = stats.kruskal(*groupes_par_categorie)
    print("Résultats du test de Kruskal-Wallis :")
    print(f"Statistique de test : {kruskal_stat}")
    print(f"P-valeur : {kruskal_p_value}")
else:
    print("\nConditions pour l'ANOVA satisfaites. Vous pouvez effectuer l'ANOVA.")


Test de Shapiro-Wilk pour la catégorie 0:
Statistique de test : 0.9365479350090027
P-valeur : 0.0
L'échantillon ne semble pas provenir d'une distribution normale (l'hypothèse nulle est rejetée)

Test de Shapiro-Wilk pour la catégorie 1:
Statistique de test : 0.9906350374221802
P-valeur : 0.0
L'échantillon ne semble pas provenir d'une distribution normale (l'hypothèse nulle est rejetée)

Test de Shapiro-Wilk pour la catégorie 2:
Statistique de test : 0.6679465174674988
P-valeur : 0.0
L'échantillon ne semble pas provenir d'une distribution normale (l'hypothèse nulle est rejetée)

Test d'homogénéité des variances (Test de Levene) :
Statistique de test : 26975.654952034234
P-valeur : 0.0
L'homogénéité des variances n'est pas vérifiée (l'hypothèse nulle est rejetée)

Conditions pour l'ANOVA non remplies. Effectuez le test de Kruskal-Wallis.
Résultats du test de Kruskal-Wallis :
Statistique de test : 78458.43837285662
P-valeur : 0.0


Les résultats des tests sont assez clairs :

Les tests de Shapiro-Wilk pour chaque catégorie montrent que l'hypothèse de normalité est rejetée pour toutes les catégories, car les p-valeurs sont très proches de zéro. Cela signifie que l'âge n'est pas distribué normalement dans chaque catégorie.

Le test d'homogénéité des variances (Test de Levene) montre que l'hypothèse d'homogénéité des variances est rejetée car la p-valeur est très proche de zéro. Cela signifie que les variances de l'âge dans chaque catégorie sont significativement différentes.

En conséquence, les conditions pour l'ANOVA ne sont pas remplies, et le test de Kruskal-Wallis est effectué pour la comparaison des médianes.

Le test de Kruskal-Wallis montre également des résultats significatifs avec une p-valeur proche de zéro, indiquant que les médianes de l'âge dans les catégories sont différentes.

Dans l'ensemble, ces résultats suggèrent que l'âge a une influence significative sur la catégorie de livres achetés, mais il ne suit pas une distribution normale et les variances ne sont pas homogènes.
Les résultats du test de Kruskal-Wallis montrent une statistique de test élevée (78458.44) avec une p-valeur très proche de zéro (0.0). Cela signifie que les médianes des tranches d'âge diffèrent de manière significative entre les catégories de livres achetés.

En d'autres termes, il y a une relation statistiquement significative entre l'âge des clients et la catégorie de livres qu'ils achètent.

